In [1]:
import glob, os, json
import time
import pickle
import solver
import traceback
import signal

In [2]:
timeHeuristicsOnly = True
timeSymmetricOnly = True

# Parameters
timingIterations = 2
featureTimeout = 300
heldKarpTimeout = 600

# Paths
tspLibPath = "../data/tsplib/tsp/"
atspLibPath = "../data/tsplib/atsp/"
generatedPath = "../data/generated/"

# Prep
nLowestEdgeCosts = 10
nGeneratedSolutions = 100

# Heuristics
annealingStartTemp = 1000000
annealingEndTemp = 0.01
annealingIterations = 100000

tabuMaxCandidates = 1000
tabuSizeMax = 100
tabuNoImprovements = 20
tabuTimeLimit = 30

graspNoImprovements = 10000
graspIterations = 100000
graspAlpha = 0.2
graspTimeLimit = 30

geneticGenerations = 300000
geneticTimeLimit = 30

antCount = 5
antIterations = 10000
antRepetitions = 1
antTimeLimit = 30

# Multiprocessing
processes = 16
chunksize = 3

In [3]:
def loadTSPInstances(path, extension):
    instances = []
    for file in glob.glob(path + "*." + extension):
        try:
            tsp = solver.loadTSPLib(file)
            name = os.path.basename(file)
            if not tsp:
                print("Invalid file at " + name)
                continue

            tsp.setName(name)
            instances.append(tsp)
        except:
            traceback.print_exc()
    
    return instances

def loadGeneratedInstances(path):
    instances = []
    for file in glob.glob(path + "*.pytsp"):
        try:
            tspFile = open(file, "rb")
            tsp = pickle.load(tspFile)
            name = os.path.basename(file)
            
            if not tsp:
                print("Invalid file at " + name)
                continue

            tsp.setName(name)
            instances.append(tsp)
            tspFile.close()
        except:
            traceback.print_exc()
            
    return instances

In [4]:
# Load all problem instances
tspLibInstances = loadTSPInstances(tspLibPath, "tsp")
atspLibInstances = loadTSPInstances(atspLibPath, "atsp")
generatedInstances = loadGeneratedInstances(generatedPath)

In [5]:
tspGeneratedInstances = []
atspGeneratedInstances = []

for instance in generatedInstances:
    if instance.isAsymmetric():
        atspGeneratedInstances.append(instance)
    else:
        tspGeneratedInstances.append(instance)

In [ ]:
def timeoutHandler(signum, frame):
    print("Timeout")
    raise TimeoutError()

def measureFeature(name, deterministic, tsp, data, features, featureFunction, storeResult=True, cleanup=lambda: None, customTimeout=None):    
    timeout = featureTimeout
    if customTimeout:
        timeout = customTimeout
    
    signal.signal(signal.SIGALRM, timeoutHandler)
    signal.alarm(timeout)
    
    value = None
    
    start = 0
    end = 0
    
    try:
        start = time.time()
        value = featureFunction(tsp, data)
        end = time.time()
    except TimeoutError:
        cleanup()
        # Set timeout value to -1
        start = 1
        end = 0
        
    signal.alarm(0)
    
    storedValue = value
    if type(storedValue).__module__=='numpy': # if value is numpy.*: > to python list
        storedValue = storedValue.tolist()
    
    if storeResult:
        if deterministic:
            features[name] = storedValue
        else:
            valueName = name + "Values"
            if valueName not in features:
                features[valueName] = []

            features[valueName].append(storedValue)
        
    timeName = name + "Times"
    
    if timeName not in features:
        features[timeName] = []
    
    features[timeName].append(end - start)
    
    return value

In [ ]:
def stripNumpyDict(dictionary):
    for key in dictionary:
        value = dictionary[key]
        if isinstance(value, dict):
            stripNumpyDict(value)
        if isinstance(value, list):
            dictionary[key] = stripNumpyArray(value)
        elif isinstance(value, numpy.ndarray):
            dictionary[key] = stripNumpyArray(value.tolist())
        elif type(value).__module__=='numpy': # if value is numpy.*: > to python list
            dictionary[key] = value.item()
            
def stripNumpyArray(array):
    newArray = []
    for item in array:
        if isinstance(item, list):
            newArray.append(stripNumpyArray(item))
        elif isinstance(item, numpy.ndarray):
            newArray.append(stripNumpyArray(item.tolist()))
        elif type(item).__module__=='numpy': # if value is numpy.*: > to python list
            newArray.append(item.item())
        else:
            newArray.append(item)
            
    return newArray

In [ ]:
from network.features import *
from network.simpleFeatures import *
from network.complexFeatures import *
from network.heuristicFeatures import *
import json

def generateSimpleFeatures(tsp, features, n):
    simple = {}
    for i in range(n):
        print("Starting Simple Features %d for %s" % (i + 1, tsp.getName()))
        measureFeature("numberVertices", True, tsp, None, simple, lambda tsp, _: numberVertices(tsp))
        
        vertexCosts = measureFeature("vertexCostPrep", True, tsp, None, simple, lambda tsp, _: buildVertexCosts(tsp), False)
        data = {"vertexCosts": vertexCosts}
        measureFeature("lowestVertexCost", True, tsp, data, simple, lambda _, data: lowestVertexCost(data["vertexCosts"]))
        measureFeature("highestVertexCost", True, tsp, data, simple, lambda _, data: highestVertexCost(data["vertexCosts"]))
        measureFeature("averageVertexCost", True, tsp, data, simple, lambda _, data: averageVertexCost(data["vertexCosts"]))
        measureFeature("standardDeviationVertexCost", True, tsp, data, simple, lambda _, data: standardDeviationVertexCost(data["vertexCosts"]))
        measureFeature("medianVertexCost", True, tsp, data, simple, lambda _, data: medianVertexCost(data["vertexCosts"]))
        
        measureFeature("sumCostNearestNeighbor", True, tsp, None, simple, lambda tsp, _: sumCostNearestNeighbor(tsp))
        measureFeature("numberEdges", True, tsp, None, simple, lambda tsp, _: numberEdges(tsp))
        measureFeature("lowestEdgeCost", True, tsp, None, simple, lambda tsp, _: lowestEdgeCost(tsp))
        measureFeature("highestEdgeCost", True, tsp, None, simple, lambda tsp, _: highestEdgeCost(tsp))
        measureFeature("averageEdgeCost", True, tsp, None, simple, lambda tsp, _: averageEdgeCost(tsp))
        measureFeature("standardDeviationEdgeCost", True, tsp, None, simple, lambda tsp, _: standardDeviationEdgeCost(tsp))
        measureFeature("medianEdgeCost", True, tsp, None, simple, lambda tsp, _: medianEdgeCost(tsp))
        measureFeature("sumNLowestEdgeCost", True, tsp, None, simple, lambda tsp, _: sumNLowestEdgeCost(tsp, nLowestEdgeCosts))
        
        print("Ending Simple Features %d for %s" % (i + 1, tsp.getName()))
        
    features["simpleFeatures"] = simple
    
def generateComplexFeatures(tsp, features, n):
    complexFeatures = {}
    for i in range(n):
        print("Starting Complex Features %d for %s" % (i + 1, tsp.getName()))
        measureFeature("averageShortestPathGeodesicDistance", True, tsp, None, complexFeatures, lambda tsp, _: averageShortestPathGeodesicDistance(tsp))
        measureFeature("averageGeodesicDistance", True, tsp, None, complexFeatures, lambda tsp, _: averageGeodesicDistance(tsp))
        
        globalEfficiencyValue = measureFeature("globalEfficiency", True, tsp, None, complexFeatures, lambda tsp, _: globalEfficiency(tsp))
        data = {"globalEfficiency": globalEfficiencyValue}
        measureFeature("harmonicMeanGeodesicDistance", True, tsp, None, complexFeatures, lambda tsp, _: harmonicMeanGeodesicDistance(tsp))
        
        measureFeature("networkVulnerability", True, tsp, data, complexFeatures, lambda tsp, data: networkVulnerability(tsp, data["globalEfficiency"]))
        
        measureFeature("clusteringCoefficientTransitivity", True, tsp, None, complexFeatures, lambda tsp, _: clusteringCoefficientTransitivity(tsp))
        measureFeature("alternateClusteringCoefficient", True, tsp, None, complexFeatures, lambda tsp, _: alternateClusteringCoefficient(tsp))
        
        connectedEdgeCountsValue = measureFeature("connectedEdgeCountsPrep", True, tsp, None, complexFeatures, lambda tsp, _: connectedEdgeCounts(tsp), False)
        data = {"connectedEdgeCounts": connectedEdgeCountsValue}
        measureFeature("weightedClusteringCoefficient", True, tsp, data, complexFeatures, lambda tsp, data: weightedClusteringCoefficient(tsp, data["connectedEdgeCounts"]))
        measureFeature("networkCyclicCoefficient", True, tsp, data, complexFeatures, lambda tsp, data: networkCyclicCoefficient(tsp, data["connectedEdgeCounts"]))
        measureFeature("maxVertexDegree", True, tsp, data, complexFeatures, lambda _, data: maxVertexDegree(data["connectedEdgeCounts"]))
        measureFeature("edgeDegreeCorrelation", True, tsp, data, complexFeatures, lambda tsp, data: edgeDegreeCorrelation(tsp, data["connectedEdgeCounts"]))
        
        measureFeature("entropyDegreeDistribution", True, tsp, None, complexFeatures, lambda tsp, _: entropyDegreeDistribution(tsp))
        measureFeature("targetEntropy", True, tsp, None, complexFeatures, lambda tsp, _: targetEntropy(tsp))
        measureFeature("vertexParticipationCoefficient", True, tsp, None, complexFeatures, lambda tsp, _: vertexParticipationCoefficient(tsp))
        measureFeature("edgeReciprocity", True, tsp, None, complexFeatures, lambda tsp, _: edgeReciprocity(tsp))
        measureFeature("adjacencyCorrelationCoefficient", True, tsp, None, complexFeatures, lambda tsp, _: adjacencyCorrelationCoefficient(tsp))
        
        print("Ending Complex Features %d for %s" % (i + 1, tsp.getName()))
        
    features["complexFeatures"] = complexFeatures
    
def generateHeuristicFeatures(tsp, features, n):
    heuristicFeatures = {}
    for i in range(n):
        print("Starting Heuristic Features %d for %s" % (i + 1, tsp.getName()))
        
        random = measureFeature("randomSolutionsPrep", False, tsp, None, heuristicFeatures, lambda tsp, _: randomSolutions(tsp, nGeneratedSolutions), False)
        data = {"randomSolutions": random}
        randomNeighbor = measureFeature("randomNeighborSolutionsPrep", False, tsp, data, heuristicFeatures, lambda tsp, data: list(map(lambda solution: neighborSolutions(tsp, solution, nGeneratedSolutions), data["randomSolutions"])), False)
        
        # Remove most of randomNeighbors from features
        # heuristicFeatures["randomNeighborSolutionsPrep"] = heuristicFeatures["randomNeighborSolutionsPrep"][:min(5, nGeneratedSolutions)]
        
        data = {"randomSolutions": random, "randomNeighborSolutions": randomNeighbor}
        measureFeature("proportionsNeighborsWithBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: proportionsNeighborsWithBetterSolution(tsp, data["randomSolutions"], data["randomNeighborSolutions"]))
        measureFeature("averageRatioNeighborsWithBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: averageRatioNeighborsWithBetterSolution(tsp, data["randomSolutions"], data["randomNeighborSolutions"]))
        measureFeature("randomNeighborSolutionQuality", False, tsp, data, heuristicFeatures, lambda tsp, data: randomNeighborSolutionQuality(tsp, data["randomSolutions"], data["randomNeighborSolutions"]))
        
        greedy = measureFeature("greedySolutionsPrep", False, tsp, None, heuristicFeatures, lambda tsp, _: greedySolutions(tsp, nGeneratedSolutions), False)
        data = {"randomSolutions": random, "greedySolutions": greedy}
        measureFeature("greedySolutionQuality", False, tsp, data, heuristicFeatures, lambda tsp, data: greedySolutionQuality(tsp, data["randomSolutions"], data["greedySolutions"]))
        measureFeature("averageRatioGreedySolution", False, tsp, data, heuristicFeatures, lambda tsp, data: averageRatioGreedySolution(tsp, data["randomSolutions"], data["greedySolutions"]))
        
        parents = measureFeature("parentSolutionPairsPrep", False, tsp, data, heuristicFeatures, lambda tsp, data: solutionPairs(tsp, data["randomSolutions"], nGeneratedSolutions), False)
        data = {"parentSolutionPairs": parents}
        children = measureFeature("childrenSolutionPairsPrep", False, tsp, data, heuristicFeatures, lambda tsp, data: childrenSolutionPairs(tsp, data["parentSolutionPairs"]), False)
        
        data = {"parentSolutionPairs": parents, "childrenSolutionPairs": children}
        measureFeature("proportionOffspringBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: proportionOffspringBetterSolution(tsp, data["parentSolutionPairs"], data["childrenSolutionPairs"]))
        measureFeature("averageRatioOffspringBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: averageRatioOffspringBetterSolution(tsp, data["parentSolutionPairs"], data["childrenSolutionPairs"]))
        measureFeature("averageTimesOffspringBetterSolution", False, tsp, data, heuristicFeatures, lambda tsp, data: averageTimesOffspringBetterSolution(tsp, data["parentSolutionPairs"], data["childrenSolutionPairs"]))
        
        data = {"greedySolutions": greedy}
        greedyEdges = measureFeature("greedyEdgesPrep", False, tsp, data, heuristicFeatures, lambda tsp, data: edges(tsp, data["greedySolutions"]), False)
        data = {"greedyEdges": greedyEdges}
        measureFeature("averageSharedEdges", False, tsp, data, heuristicFeatures, lambda tsp, data: averageSharedEdges(tsp, data["greedyEdges"]))
        measureFeature("relativeFrequencyCommonGreedyEdge", False, tsp, data, heuristicFeatures, lambda tsp, data: relativeFrequencyCommonGreedyEdge(tsp, data["greedyEdges"]))
        
        print("Ending Heuristic Features %d for %s" % (i + 1, tsp.getName()))
        
    features["heuristicFeatures"] = heuristicFeatures
        
def generateFeatures(tsp, features, n):
    generateSimpleFeatures(tsp, features, n)
    generateComplexFeatures(tsp, features, n)
    generateHeuristicFeatures(tsp, features, n)

In [ ]:
from solver.utilities import calculateCost

def measureHeuristicCost(name, tsp, features):
    valueName = name + "Values"
    if valueName in features:
        latestTour = features[valueName][-1]
        cost = calculateCost(numpy.array(latestTour), tsp)
        
        costName = name + "Costs"
        if costName not in features:
            features[costName] = []

        features[costName].append(cost)

In [ ]:
import heuristics

def generateSimulatedAnnealing(tsp):
    return heuristics.solveSimAnneal(tsp, annealingStartTemp, annealingEndTemp, annealingIterations, False)

def generateTabu(tsp):
    return heuristics.solveTabu(tsp, tabuMaxCandidates, tabuSizeMax, tabuNoImprovements, tabuTimeLimit, False)

def generateGrasp(tsp):
    return heuristics.solveGrasp(tsp, graspNoImprovements, graspIterations, graspAlpha, graspTimeLimit, False)

def generateGenetic(tsp):
    return heuristics.solveGenetic(tsp, geneticGenerations, geneticTimeLimit, False)

def generateAntColony(tsp):
    return heuristics.solveAntColony(tsp, antCount, antIterations, antRepetitions, antTimeLimit, False)

def generateHeuristics(tsp, features, n):
    heuristics = {}
    for i in range(n):
        print("Starting Simulated Annealing %d for %s" % (i + 1, tsp.getName()))
        measureFeature("simulatedAnnealing", False, tsp, None, heuristics, lambda tsp, _: generateSimulatedAnnealing(tsp))
        measureHeuristicCost("simulatedAnnealing", tsp, heuristics)
        
        print("Starting Tabu %d for %s" % (i + 1, tsp.getName()))
        measureFeature("tabu", False, tsp, None, heuristics, lambda tsp, _: generateTabu(tsp))
        measureHeuristicCost("tabu", tsp, heuristics)
        
        print("Starting GRASP %d for %s" % (i + 1, tsp.getName()))
        measureFeature("grasp", False, tsp, None, heuristics, lambda tsp, _: generateGrasp(tsp))
        measureHeuristicCost("grasp", tsp, heuristics)
        
        print("Starting Genetic %d for %s" % (i + 1, tsp.getName()))
        measureFeature("genetic", False, tsp, None, heuristics, lambda tsp, _: generateGenetic(tsp))
        measureHeuristicCost("genetic", tsp, heuristics)
        
        print("Starting Ant Colony %d for %s" % (i + 1, tsp.getName()))
        measureFeature("antColony", False, tsp, None, heuristics, lambda tsp, _: generateAntColony(tsp))
        measureHeuristicCost("antColony", tsp, heuristics)
        
    features["heuristics"] = heuristics

In [ ]:
def generateHeldKarp(tsp, features):
    print("Generating Held-Karp for %s" % (tsp.getName()))
    measureFeature("heldKarp", True, tsp, None, features, lambda tsp, _: solver.findLowerBound(tsp, times=1), customTimeout=heldKarpTimeout)

In [ ]:
def loadOptimal(tsp, features, isAsymmetric):
    filename = tspLibPath
    if isAsymmetric:
        filename = atspLibPath
        
    filename += os.path.splitext(tsp.getName())[0] + ".opt.tour"
    
    print(filename)
    
    if os.path.isfile(filename):
        optimalTour = solver.loadTSPLibTour(filename, tsp)
        if optimalTour:
            features["optimalTour"] = optimalTour.tour
            features["optimalCost"] = optimalTour.cost
        else:
            print("Failed to load optimal tour for " + filename)
            
def generateMetadata(tsp, features):
    metadata = {}
    
    metadata["isAsymmetric"] = tsp.isAsymmetric()
    
    features["metadata"] = metadata

In [ ]:
def writeFeatures(features, filename):
    stripNumpyDict(features)
    
    print("Writing File %s" % (filename))
    
    with open(filename, "w") as file:
        json.dump(features, file, indent="\t")
        
    print("Finished writting")

In [ ]:
def generateAll(tsp, isTSPLib, n, isAsymmetric):
    filename = os.path.splitext(tsp.getName())[0]
    path = "../data/features/tsplib/"
    
    if not isTSPLib:
        path = "../data/features/generated/"
        
    path += filename + ".json"
    
    features = {}
    
    try:
        loadOptimal(tsp, features, isAsymmetric)
        generateMetadata(tsp, features)
        generateHeldKarp(tsp, features)
        if not timeHeuristicsOnly:
            generateFeatures(tsp, features, n)
        generateHeuristics(tsp, features, n)
    except:
        traceback.print_exc()
        print("Failure in %s" % (path))
                
    writeFeatures(features, path)

In [ ]:
from multiprocessing import Pool

class TSPData(object):
    def __init__(self, isTSPLib, n, isAsymmetric):
        self.isTSPLib = isTSPLib
        self.n = n
        self.isAsymmetric = isAsymmetric
    def __call__(self, tsp):
        featuresFromTSP(tsp, self.isTSPLib, self.n, self.isAsymmetric)

def featuresFromTSP(tsp, isTSPLib, n, isAsymmetric):
    print("Examining %s" % (tsp.getName()))

    generateAll(tsp, isTSPLib, n, isAsymmetric)
    
startTime = time.time()

# TSPLib
with Pool(processes=processes) as pool:
    pool.map(TSPData(True, timingIterations, True), tspLibInstances, chunksize)
    
print("Completed Symmetric TSPLib Processing")

if not timeSymmetricOnly:
    with Pool(processes=processes) as pool:
        pool.map(TSPData(True, timingIterations, False), atspLibInstances, chunksize)

    print("Completed Asymmetric TSPLib Processing")

# Generated
with Pool(processes=processes) as pool:
    pool.map(TSPData(False, timingIterations, False), tspGeneratedInstances, chunksize)

if not timeSymmetricOnly:
    with Pool(processes=processes) as pool:
        pool.map(TSPData(False, timingIterations, True), atspGeneratedInstances, chunksize)

    print("Completed Generated Processing")

endTime = time.time()

print("Completed in %d" % (endTime - startTime))